In [3]:
#instead of training my own cnn from scratch, i'm going to fine tune the resnet50 pretrained model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
baseModel = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

2022-07-20 15:12:16.549566: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-20 15:12:16.549592: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shivam-IdeaPad-Gaming-3-15ARH05): /proc/driver/nvidia/version does not exist
2022-07-20 15:12:16.549812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


94765736/94765736 [==============================] - 19s 0us/step


In [22]:
import tensorflow as tf

In [4]:
baseModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [6]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(120, activation="softmax")(headModel)

In [7]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [13]:
from keras.preprocessing.image import ImageDataGenerator
trainAug = ImageDataGenerator(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")

In [14]:
valAug = ImageDataGenerator()

In [16]:
#normailize the data
import numpy as np
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [18]:
trainGen = trainAug.flow_from_directory(
    'dataset/organized/training',
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=32)

Found 10222 images belonging to 120 classes.


In [19]:
valGen = valAug.flow_from_directory(
    'dataset/organized/validating',
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=32)

Found 2045 images belonging to 120 classes.


In [20]:
#freezing the base layers
for layer in baseModel.layers:
    layer.trainable = False

In [28]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, ),metrics=["accuracy"])

In [29]:
model.fit(x=trainGen, validation_data=valGen, epochs=25, steps_per_epoch=10222//32, validation_steps=2045//32)

Epoch 1/25
319/319 [==============================] - 566s 2s/step - loss: 5.0080 - accuracy: 0.0214 - val_loss: 4.3353 - val_accuracy: 0.1171
Epoch 2/25
319/319 [==============================] - 557s 2s/step - loss: 4.2837 - accuracy: 0.0773 - val_loss: 3.5558 - val_accuracy: 0.2540
Epoch 3/25
319/319 [==============================] - 557s 2s/step - loss: 3.7381 - accuracy: 0.1562 - val_loss: 2.8535 - val_accuracy: 0.3666
Epoch 4/25
319/319 [==============================] - 545s 2s/step - loss: 3.3342 - accuracy: 0.2062 - val_loss: 2.4473 - val_accuracy: 0.4464
Epoch 5/25
319/319 [==============================] - 546s 2s/step - loss: 3.0161 - accuracy: 0.2558 - val_loss: 2.1114 - val_accuracy: 0.5174
Epoch 6/25
319/319 [==============================] - 557s 2s/step - loss: 2.7868 - accuracy: 0.3086 - val_loss: 1.9037 - val_accuracy: 0.5476
Epoch 7/25
319/319 [==============================] - 521s 2s/step - loss: 2.6095 - accuracy: 0.3296 - val_loss: 1.7345 - val_accuracy: 0.5952

In [30]:
model.fit(x=trainGen, validation_data=valGen, epochs=25, steps_per_epoch=10222//32, validation_steps=2045//32)

Epoch 1/25
319/319 [==============================] - 555s 2s/step - loss: 1.5354 - accuracy: 0.5656 - val_loss: 0.8783 - val_accuracy: 0.7634
Epoch 2/25
319/319 [==============================] - 582s 2s/step - loss: 1.5359 - accuracy: 0.5662 - val_loss: 0.8726 - val_accuracy: 0.7718
Epoch 3/25
319/319 [==============================] - 543s 2s/step - loss: 1.4925 - accuracy: 0.5746 - val_loss: 0.8540 - val_accuracy: 0.7684
Epoch 4/25
319/319 [==============================] - 567s 2s/step - loss: 1.4644 - accuracy: 0.5867 - val_loss: 0.8307 - val_accuracy: 0.7812
Epoch 5/25
319/319 [==============================] - 601s 2s/step - loss: 1.4399 - accuracy: 0.5871 - val_loss: 0.8100 - val_accuracy: 0.7857
Epoch 6/25
319/319 [==============================] - 596s 2s/step - loss: 1.4261 - accuracy: 0.5931 - val_loss: 0.8106 - val_accuracy: 0.7793
Epoch 7/25
319/319 [==============================] - 595s 2s/step - loss: 1.3856 - accuracy: 0.6069 - val_loss: 0.7852 - val_accuracy: 0.7892

In [32]:
trainGen.reset()
valGen.reset()

In [33]:
sample = pd.read_csv('dataset/sample_submission.csv')
sample

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10352,ffeda8623d4eee33c6d1156a2ecbfcf8,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
10353,fff1ec9e6e413275984966f745a313b0,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
10354,fff74b59b758bbbf13a5793182a9bbe4,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
10355,fff7d50d848e8014ac1e9172dc6762a3,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [34]:
submission = pd.DataFrame(columns=[sample.columns])

In [35]:
ids = sample['id'].values

In [38]:
for i in range(0, len(ids)):
    row = [ids[i]]
    test_image = tf.keras.utils.load_img('dataset/test/' + ids[i] + '.jpg', target_size=(224,224))
    test_image = tf.keras.utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    result = result[0].tolist()
    row = row + result
    submission.loc[i] = row

1/1 [==============================] - 0s 70ms/step


1/1 [==============================] - 0s 70ms/step


1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 86ms/step


...

1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 74ms/step


In [39]:
submission

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,5.128007e-07,7.594399e-02,1.454555e-05,8.797891e-07,8.500333e-07,8.314870e-06,1.284087e-05,5.470304e-07,1.203568e-05,...,2.048188e-05,2.459525e-05,9.934408e-07,1.469098e-06,7.689920e-07,4.605057e-03,8.309951e-07,1.421078e-05,5.299250e-07,3.008681e-05
1,00102ee9d8eb90812350685311fe5890,4.072734e-08,1.272020e-09,2.830131e-09,1.341953e-09,3.014417e-06,9.228901e-08,9.037331e-10,3.789479e-07,7.559406e-09,...,1.842449e-08,1.963251e-06,1.934751e-09,9.137135e-09,1.786706e-09,8.703769e-11,4.022669e-07,2.785071e-08,1.139601e-08,1.323262e-08
2,0012a730dfa437f5f3613fb75efcd4ce,2.575582e-06,4.878494e-05,2.035555e-03,2.608222e-04,4.831377e-04,1.095938e-04,1.789198e-07,3.668771e-07,3.890773e-05,...,1.943485e-05,1.678373e-05,2.013715e-05,8.171931e-04,2.159036e-02,4.127577e-04,1.449208e-05,1.785921e-02,3.507530e-05,3.555446e-06
3,001510bc8570bbeee98c8d80c8a95ec1,3.668698e-03,1.929134e-04,5.978132e-04,2.589354e-03,2.178056e-03,7.631264e-04,3.089061e-05,2.514506e-05,7.992100e-03,...,1.282867e-03,4.602823e-05,3.869810e-03,1.708079e-04,1.148104e-03,8.881507e-06,5.551672e-05,7.114429e-03,2.204885e-05,1.812627e-03
4,001a5f3114548acdefa3d4da05474c2e,1.818872e-03,1.089423e-04,5.752448e-06,1.167618e-06,1.281810e-07,6.948091e-07,1.751915e-03,3.570443e-09,1.223016e-06,...,2.219645e-06,4.975731e-07,3.923918e-08,6.289699e-08,6.652655e-08,7.428803e-06,2.069198e-04,1.366141e-06,1.401442e-05,3.277656e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10352,ffeda8623d4eee33c6d1156a2ecbfcf8,3.962783e-07,4.584563e-04,8.938125e-07,9.684355e-04,6.295792e-06,4.710306e-06,3.501715e-07,2.387380e-07,1.410530e-06,...,5.120262e-03,1.476791e-06,4.080864e-06,2.501038e-06,3.237564e-05,2.663110e-06,2.666930e-06,1.503012e-05,1.257006e-04,1.269063e-06
10353,fff1ec9e6e413275984966f745a313b0,9.253911e-05,1.885833e-04,1.450954e-04,2.652339e-02,2.578693e-03,1.971344e-04,6.044066e-05,6.371819e-04,1.170515e-02,...,3.422628e-05,1.679435e-04,8.172301e-03,4.720246e-02,1.576261e-01,2.374604e-04,9.747533e-05,6.290111e-02,1.414881e-03,2.622503e-04
10354,fff74b59b758bbbf13a5793182a9bbe4,1.086321e-06,7.878515e-05,4.177294e-03,1.586327e-04,2.179451e-05,7.441451e-05,4.898571e-04,7.316293e-05,1.782242e-05,...,1.585539e-06,5.881022e-05,1.719286e-04,9.648811e-06,3.743345e-05,2.470659e-04,3.730811e-07,1.102205e-03,1.948884e-04,3.749494e-04
10355,fff7d50d848e8014ac1e9172dc6762a3,1.994107e-03,1.550383e-04,6.847210e-07,4.518572e-06,7.668409e-07,2.557211e-06,8.892262e-04,1.803420e-08,1.016960e-05,...,9.024519e-05,1.458752e-05,3.199547e-07,1.344646e-07,2.047426e-06,1.723513e-05,7.569502e-04,6.867107e-06,1.043484e-04,3.773311e-02


In [40]:
submission.to_csv('submission3.csv', index=False)

In [43]:
import pickle
saved = pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://b7a4ad1c-7d8e-42b6-9739-2e2d0dd9dfe6/assets


INFO:tensorflow:Assets written to: ram://b7a4ad1c-7d8e-42b6-9739-2e2d0dd9dfe6/assets


In [44]:
loaded = pickle.load(open('model.pkl', 'rb'))

In [46]:
test_image = tf.keras.utils.load_img('dataset/test/000621fb3cbb32d8935728e48679680e.jpg', target_size=(224,224))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
result

1/1 [==============================] - 0s 480ms/step


array([[5.12800682e-07, 7.59439915e-02, 1.45455460e-05, 8.79789127e-07,
        8.50033302e-07, 8.31486977e-06, 1.28408692e-05, 5.47030425e-07,
        1.20356799e-05, 3.08136921e-04, 3.18476090e-08, 2.68371718e-04,
        3.13083973e-07, 8.25442523e-02, 7.73944266e-05, 8.90362207e-06,
        1.87635014e-04, 6.26618714e-07, 6.48605404e-03, 1.99327860e-06,
        1.25182538e-07, 6.58104182e-06, 3.22490610e-06, 2.98278383e-03,
        5.64787909e-03, 1.83688621e-06, 3.34764582e-06, 1.27255364e-04,
        7.12344445e-07, 8.70633739e-05, 1.17906886e-04, 9.61771514e-03,
        2.02980824e-03, 5.66847529e-03, 2.99363840e-08, 1.55020480e-05,
        3.52190426e-07, 9.32335013e-07, 5.63322473e-08, 2.10146641e-06,
        2.99652526e-03, 6.96135103e-05, 1.09431548e-05, 1.01135614e-04,
        4.55549480e-06, 5.07175719e-06, 1.01275900e-05, 5.73419698e-07,
        8.07973777e-09, 3.79012735e-03, 1.44749729e-05, 3.91587764e-06,
        6.22744039e-02, 3.32189211e-06, 4.22947835e-07, 1.449398